In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split

/usr/local/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:45: FutureWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS will be built by the Apple Clang compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you won't need to install the gcc compiler anymore.
Instead of that, you'll need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", FutureWarning)


In [2]:
print('Load training data...')
df_x_train = pd.read_csv('X_train.csv', header=0, index_col = 0)
df_y_train = pd.read_csv('y_train.csv', header=0, index_col = 0)

Load training data...


In [3]:
df_x_train.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x990,x991,x992,x993,x994,x995,x996,x997,x998,x999
id,,,,,,,,,,,,,,,,,,,,,
0,-4.524480,-0.718917,0.827537,-2.336564,1.905992,-1.424985,-5.622933,-0.739429,0.334367,-0.245799,...,-3.214350,-4.083583,-1.240234,1.581522,-3.147444,0.423618,2.387999,1.784247,-1.689361,-1.586569
1,-0.561814,-0.115757,-0.113303,-0.322508,-0.080855,0.042634,-0.319990,-0.066997,0.281196,-0.064463,...,-0.262083,-0.437542,0.300902,0.502415,-0.537463,0.455991,-0.378800,-0.536470,-0.810315,-0.021378
2,-0.547026,-0.045593,1.016072,-0.068002,-0.670472,-0.551299,-0.550926,0.393147,1.022467,-0.113551,...,0.064608,-0.361322,-0.440028,0.278972,-0.570960,-0.708099,-0.025025,0.552631,-1.365591,-0.584266
3,-1.939258,-0.284554,1.276007,-0.500731,1.088817,-0.897736,-1.530660,-0.952914,1.157809,0.149595,...,-2.341105,-1.663250,-1.224091,0.617387,-0.964099,-0.034949,0.157197,0.137123,-0.165070,-0.740363
4,-0.386835,-0.143997,0.506509,-0.648928,-0.614121,0.211504,0.025600,-0.272372,-0.215564,0.070822,...,0.083148,0.568495,0.050980,0.317415,-0.163551,-0.240090,-0.270020,-0.296239,-0.722527,0.986404


In [4]:
df_y_train.head()

,y
id,
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0


In [5]:
print('Splitting into training and validation dataset')
X = df_x_train.values
y = df_y_train['y'].values


Splitting into training and validation dataset


In [32]:
best = 0

for i in range(100):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2) # , random_state = 19960503)
    # create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    # specify your configurations as a dict
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclassova',
        'is_unbalance': True,
        #'metric': {'l1', 'l2'},
        'num_class': 3,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.2,
        'bagging_fraction': 0.8,
        'verbose': 0,
    }

    def custom_accuracy(preds, train_data):
        labels = train_data.get_label()
        n = len(labels)
        results = []
        for i in range(n):
            results.append(np.argmax([preds[i], preds[n + i], preds[2*n + i]]))
        return 'BMAC', balanced_accuracy_score(labels, results), True

    print('Start training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    feval=custom_accuracy,
                    valid_sets={lgb_train, lgb_eval},
                    early_stopping_rounds=10)
    
    y_val_p = gbm.predict(X_val, num_iteration=gbm.best_iteration)
    
    result = balanced_accuracy_score(y_val, [np.argmax(x) for x in y_val_p])
    if result > best:
        print("New Best {}".format(result))
        best = result
        y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        f = open("submission{0:.4f}.csv".format(best), "w")
        f.write("id,y\n")
        for i,x in enumerate(y_pred):
            f.write("{},{}\n".format(i,np.argmax(x)))
        f.close()

Start training...
[1]	training's multi_logloss: 0.665948	training's BMAC: 0.770416	valid_1's multi_logloss: 0.672044	valid_1's BMAC: 0.582186
Training until validation scores don't improve for 10 rounds.
[2]	training's multi_logloss: 0.640837	training's BMAC: 0.822891	valid_1's multi_logloss: 0.653342	valid_1's BMAC: 0.584033
[3]	training's multi_logloss: 0.617824	training's BMAC: 0.844531	valid_1's multi_logloss: 0.635327	valid_1's BMAC: 0.598254
[4]	training's multi_logloss: 0.596861	training's BMAC: 0.866648	valid_1's multi_logloss: 0.617972	valid_1's BMAC: 0.630469
[5]	training's multi_logloss: 0.576718	training's BMAC: 0.875258	valid_1's multi_logloss: 0.601905	valid_1's BMAC: 0.620946
[6]	training's multi_logloss: 0.558222	training's BMAC: 0.885052	valid_1's multi_logloss: 0.587691	valid_1's BMAC: 0.635601
[7]	training's multi_logloss: 0.540349	training's BMAC: 0.891776	valid_1's multi_logloss: 0.573935	valid_1's BMAC: 0.643102
[8]	training's multi_logloss: 0.523846	training's BM

[10]	training's multi_logloss: 0.488718	training's BMAC: 0.893424	valid_0's multi_logloss: 0.560686	valid_0's BMAC: 0.621109
[11]	training's multi_logloss: 0.474745	training's BMAC: 0.898298	valid_0's multi_logloss: 0.552592	valid_0's BMAC: 0.615692
[12]	training's multi_logloss: 0.461493	training's BMAC: 0.904324	valid_0's multi_logloss: 0.545068	valid_0's BMAC: 0.616639
[13]	training's multi_logloss: 0.448956	training's BMAC: 0.902651	valid_0's multi_logloss: 0.537897	valid_0's BMAC: 0.622465
[14]	training's multi_logloss: 0.436781	training's BMAC: 0.905329	valid_0's multi_logloss: 0.532072	valid_0's BMAC: 0.62289
[15]	training's multi_logloss: 0.425216	training's BMAC: 0.914011	valid_0's multi_logloss: 0.526528	valid_0's BMAC: 0.613293
[16]	training's multi_logloss: 0.414261	training's BMAC: 0.914702	valid_0's multi_logloss: 0.52281	valid_0's BMAC: 0.60993
[17]	training's multi_logloss: 0.403823	training's BMAC: 0.913319	valid_0's multi_logloss: 0.517026	valid_0's BMAC: 0.616182
[18

[23]	training's multi_logloss: 0.356854	training's BMAC: 0.923669	valid_1's multi_logloss: 0.465007	valid_1's BMAC: 0.629625
[24]	training's multi_logloss: 0.349022	training's BMAC: 0.925759	valid_1's multi_logloss: 0.460584	valid_1's BMAC: 0.639146
[25]	training's multi_logloss: 0.34188	training's BMAC: 0.925181	valid_1's multi_logloss: 0.457373	valid_1's BMAC: 0.632944
[26]	training's multi_logloss: 0.33481	training's BMAC: 0.929215	valid_1's multi_logloss: 0.453909	valid_1's BMAC: 0.631895
[27]	training's multi_logloss: 0.327771	training's BMAC: 0.933598	valid_1's multi_logloss: 0.450429	valid_1's BMAC: 0.637451
[28]	training's multi_logloss: 0.321167	training's BMAC: 0.936725	valid_1's multi_logloss: 0.447096	valid_1's BMAC: 0.637451
[29]	training's multi_logloss: 0.314888	training's BMAC: 0.938582	valid_1's multi_logloss: 0.443815	valid_1's BMAC: 0.636402
[30]	training's multi_logloss: 0.308954	training's BMAC: 0.937197	valid_1's multi_logloss: 0.440978	valid_1's BMAC: 0.633345
[3

[16]	training's multi_logloss: 0.421199	training's BMAC: 0.915382	valid_1's multi_logloss: 0.506841	valid_1's BMAC: 0.645133
[17]	training's multi_logloss: 0.411043	training's BMAC: 0.916883	valid_1's multi_logloss: 0.500667	valid_1's BMAC: 0.649496
[18]	training's multi_logloss: 0.401766	training's BMAC: 0.921503	valid_1's multi_logloss: 0.495989	valid_1's BMAC: 0.648386
[19]	training's multi_logloss: 0.392666	training's BMAC: 0.924839	valid_1's multi_logloss: 0.490811	valid_1's BMAC: 0.644373
[20]	training's multi_logloss: 0.383492	training's BMAC: 0.925084	valid_1's multi_logloss: 0.486393	valid_1's BMAC: 0.642542
[21]	training's multi_logloss: 0.375148	training's BMAC: 0.923957	valid_1's multi_logloss: 0.481472	valid_1's BMAC: 0.652106
[22]	training's multi_logloss: 0.366853	training's BMAC: 0.925689	valid_1's multi_logloss: 0.477636	valid_1's BMAC: 0.649048
[23]	training's multi_logloss: 0.35901	training's BMAC: 0.927624	valid_1's multi_logloss: 0.473027	valid_1's BMAC: 0.656937
[

[13]	training's multi_logloss: 0.454228	training's BMAC: 0.90973	valid_1's multi_logloss: 0.530336	valid_1's BMAC: 0.662
[14]	training's multi_logloss: 0.44221	training's BMAC: 0.910415	valid_1's multi_logloss: 0.522152	valid_1's BMAC: 0.668313
[15]	training's multi_logloss: 0.430938	training's BMAC: 0.912739	valid_1's multi_logloss: 0.515664	valid_1's BMAC: 0.66937
[16]	training's multi_logloss: 0.420396	training's BMAC: 0.918341	valid_1's multi_logloss: 0.508959	valid_1's BMAC: 0.665745
Early stopping, best iteration is:
[6]	training's multi_logloss: 0.55756	training's BMAC: 0.883774	valid_1's multi_logloss: 0.597012	valid_1's BMAC: 0.686541
New Best 0.6865410172767911
Start training...
[1]	training's multi_logloss: 0.666015	training's BMAC: 0.756278	valid_0's multi_logloss: 0.672335	valid_0's BMAC: 0.613792
Training until validation scores don't improve for 10 rounds.
[2]	training's multi_logloss: 0.641313	training's BMAC: 0.809507	valid_0's multi_logloss: 0.654692	valid_0's BMAC: 0

...

[7]	training's multi_logloss: 0.539209	training's BMAC: 0.897813	valid_0's multi_logloss: 0.580197	valid_0's BMAC: 0.607692
[8]	training's multi_logloss: 0.522377	training's BMAC: 0.901382	valid_0's multi_logloss: 0.568582	valid_0's BMAC: 0.617378
[9]	training's multi_logloss: 0.507032	training's BMAC: 0.907272	valid_0's multi_logloss: 0.559749	valid_0's BMAC: 0.614643
[10]	training's multi_logloss: 0.492084	training's BMAC: 0.911065	valid_0's multi_logloss: 0.549742	valid_0's BMAC: 0.608003
[11]	training's multi_logloss: 0.478373	training's BMAC: 0.910847	valid_0's multi_logloss: 0.541075	valid_0's BMAC: 0.5983
[12]	training's multi_logloss: 0.465436	training's BMAC: 0.912922	valid_0's multi_logloss: 0.533339	valid_0's BMAC: 0.596245
[13]	training's multi_logloss: 0.452619	training's BMAC: 0.915349	valid_0's multi_logloss: 0.52568	valid_0's BMAC: 0.609526
[14]	training's multi_logloss: 0.440652	training's BMAC: 0.91455	valid_0's multi_logloss: 0.518705	valid_0's BMAC: 0.605563
[15]	tr

[34]	training's multi_logloss: 0.289915	training's BMAC: 0.947233	valid_1's multi_logloss: 0.426489	valid_1's BMAC: 0.686952
[35]	training's multi_logloss: 0.284691	training's BMAC: 0.948672	valid_1's multi_logloss: 0.425324	valid_1's BMAC: 0.675256
[36]	training's multi_logloss: 0.279608	training's BMAC: 0.948378	valid_1's multi_logloss: 0.42366	valid_1's BMAC: 0.671887
[37]	training's multi_logloss: 0.274871	training's BMAC: 0.948688	valid_1's multi_logloss: 0.421914	valid_1's BMAC: 0.668054
[38]	training's multi_logloss: 0.270292	training's BMAC: 0.952149	valid_1's multi_logloss: 0.420476	valid_1's BMAC: 0.671082
[39]	training's multi_logloss: 0.265672	training's BMAC: 0.953303	valid_1's multi_logloss: 0.419223	valid_1's BMAC: 0.675102
Early stopping, best iteration is:
[29]	training's multi_logloss: 0.318533	training's BMAC: 0.941925	valid_1's multi_logloss: 0.438634	valid_1's BMAC: 0.695591
Start training...
[1]	training's multi_logloss: 0.665731	training's BMAC: 0.774647	valid_0'

[19]	training's multi_logloss: 0.391214	training's BMAC: 0.923587	valid_0's multi_logloss: 0.492248	valid_0's BMAC: 0.656884
[20]	training's multi_logloss: 0.382296	training's BMAC: 0.926479	valid_0's multi_logloss: 0.487585	valid_0's BMAC: 0.640372
[21]	training's multi_logloss: 0.373889	training's BMAC: 0.927641	valid_0's multi_logloss: 0.482588	valid_0's BMAC: 0.638939
[22]	training's multi_logloss: 0.366008	training's BMAC: 0.929845	valid_0's multi_logloss: 0.4791	valid_0's BMAC: 0.646659
[23]	training's multi_logloss: 0.358123	training's BMAC: 0.931235	valid_0's multi_logloss: 0.475381	valid_0's BMAC: 0.643337
[24]	training's multi_logloss: 0.350446	training's BMAC: 0.932044	valid_0's multi_logloss: 0.471006	valid_0's BMAC: 0.644934
[25]	training's multi_logloss: 0.343475	training's BMAC: 0.936903	valid_0's multi_logloss: 0.468058	valid_0's BMAC: 0.637213
[26]	training's multi_logloss: 0.336763	training's BMAC: 0.937252	valid_0's multi_logloss: 0.464406	valid_0's BMAC: 0.64236
[27

[11]	training's multi_logloss: 0.478864	training's BMAC: 0.899635	valid_1's multi_logloss: 0.542559	valid_1's BMAC: 0.627756
[12]	training's multi_logloss: 0.465847	training's BMAC: 0.903661	valid_1's multi_logloss: 0.535035	valid_1's BMAC: 0.625006
[13]	training's multi_logloss: 0.453242	training's BMAC: 0.908165	valid_1's multi_logloss: 0.528372	valid_1's BMAC: 0.625872
[14]	training's multi_logloss: 0.44146	training's BMAC: 0.91059	valid_1's multi_logloss: 0.520888	valid_1's BMAC: 0.64362
[15]	training's multi_logloss: 0.430534	training's BMAC: 0.91278	valid_1's multi_logloss: 0.516216	valid_1's BMAC: 0.641281
Early stopping, best iteration is:
[5]	training's multi_logloss: 0.575457	training's BMAC: 0.874295	valid_1's multi_logloss: 0.606213	valid_1's BMAC: 0.652472
Start training...
[1]	training's multi_logloss: 0.66579	training's BMAC: 0.760641	valid_0's multi_logloss: 0.672419	valid_0's BMAC: 0.58318
Training until validation scores don't improve for 10 rounds.
[2]	training's mul

[14]	training's multi_logloss: 0.44213	training's BMAC: 0.902052	valid_0's multi_logloss: 0.512786	valid_0's BMAC: 0.621064
[15]	training's multi_logloss: 0.431099	training's BMAC: 0.907667	valid_0's multi_logloss: 0.505823	valid_0's BMAC: 0.620819
[16]	training's multi_logloss: 0.420625	training's BMAC: 0.911921	valid_0's multi_logloss: 0.499193	valid_0's BMAC: 0.620156
[17]	training's multi_logloss: 0.410435	training's BMAC: 0.912277	valid_0's multi_logloss: 0.493427	valid_0's BMAC: 0.622182
[18]	training's multi_logloss: 0.40109	training's BMAC: 0.914479	valid_0's multi_logloss: 0.488938	valid_0's BMAC: 0.627301
[19]	training's multi_logloss: 0.391847	training's BMAC: 0.918508	valid_0's multi_logloss: 0.484152	valid_0's BMAC: 0.622427
[20]	training's multi_logloss: 0.382951	training's BMAC: 0.917236	valid_0's multi_logloss: 0.478425	valid_0's BMAC: 0.62094
Early stopping, best iteration is:
[10]	training's multi_logloss: 0.493186	training's BMAC: 0.897565	valid_0's multi_logloss: 0.

In [29]:
print('Load testing data...')
df_x_test = pd.read_csv('X_test.csv', header=0, index_col = 0)
# X_test = df_x_test[selected_features].values
X_test = df_x_test.values
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred
f = open("submission.csv", "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,np.argmax(x)))
f.close()

Load testing data...


In [8]:
a = [np.argmax(x) for x in y_pred]
print("0: {}".format(len([x for x in a if x == 0])))
print("1: {}".format(len([x for x in a if x == 1])))      
print("2: {}".format(len([x for x in a if x == 2])))      

0: 592
1: 2786
2: 722
